In [ ]:
!pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 31.9 MB/s eta 0:00:00


In [ ]:
import fitz

# Import the 'os' module for handling file paths and operating system interactions (not used directly here but often useful)
import os

In [ ]:
def read_text_pymupdf(path):
    """
    Extracts text from a PDF file using the PyMuPDF (fitz) library.

    Parameters:
        path (str): The file path to the PDF document.

    Returns:
        str: A single string containing all the text extracted from the PDF.
    """

    # Open the PDF document using the provided file path
    # This returns a Document object that allows access to each page
    doc = fitz.open(path)

    # Initialize an empty string to collect text from all pages
    text_results = ''

    # Loop through each page in the PDF document
    for page in doc:
        # Extract text content from the current page
        text = page.get_text()

        # Append the extracted text to the cumulative result
        text_results += text

    # Return the complete text from the PDF
    return text_results

In [ ]:

document_path="/content/plants.pdf"

docs=[read_text_pymupdf(document_path)]


In [ ]:
# Import a text splitter that breaks large texts into smaller, overlapping chunks
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Create the text splitter with settings for chunk size and overlap
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,          # Max characters per chunk
    chunk_overlap=50,         # Overlap between chunks to preserve context
    length_function=len,      # Use Python's len() to measure text length
    is_separator_regex=False, # Treat separators as plain text, not regex
    separators=[              # Preferred breakpoints for splitting
        "\n\n", "\n", " ", ".", ",",
        "\u200b", "\uff0c", "\u3001", "\uff0e", "\u3002", ""
    ],
)



In [ ]:
text_chunks=text_splitter.create_documents(docs)

In [ ]:
texts_list=[doc.page_content for doc in text_chunks]

In [ ]:
texts_list[:16]

['Aristid Lindenmayer 1925–1989\nPrzemyslaw Prusinkiewicz\nAristid Lindenmayer\nThe Algorithmic Beauty\nof Plants\nWith\nJames S. Hanan\nF. David Fracchia\nDeborah Fowler\nMartin J. M. de Boer\nLynn Mercer\nWith 150 Illustrations, 48 in Color\nThis edition of The Alogirthmic Beauty of Plants is the electronic version\nof the book that was published by Springer-Verlag, New York, in 1990 and\nreprinted in 1996. The electronic version has been produced using the orig-\ninal LATEX ﬁles and digital illustrations.\nc⃝2004 Przemyslaw Prusinkiewicz\nAll rights reserved.\nFront cover design: The roses in the foreground (Roses by D. R. Fowler,\nJ. Hanan and P. Prusinkiewicz [1990]) were modeled using L-systems. Dis-\ntributed ray-tracing with one extended light source was used to simulate\ndepth of ﬁeld. The roses were placed on a background image (photgraphy\nby G. Rossbach), which was scanned digitally and post-processed.\nPreface\nThe beauty of plants has attracted the attention of mathematic

In [ ]:
len(texts_list)

168

# Client for Runpod Infinity Server

In [ ]:
import requests
import time

# Configuration
API_URL = '<api-url>/run'
STATUS_URL = '<api-url>/status/'
API_KEY = '<api-key>'

headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {API_KEY}'
}

data = {
    "input": {
        "model": "nomic-ai/nomic-embed-text-v1.5",
        "input": "hello world"
    }
}

# Submit the job
response = requests.post(API_URL, headers=headers, json=data)
result = response.json()
job_id = result['id']

print(f"Job submitted: {job_id}")
print(f"Status: {result['status']}")

# Poll for completion
while True:
    time.sleep(2)  # Wait 2 seconds

    status_response = requests.get(f"{STATUS_URL}{job_id}", headers=headers)
    status_result = status_response.json()

    print(f"Status: {status_result['status']}")

    if status_result['status'] == 'COMPLETED':
        print("\nFinal result:")
        print(status_result)
        break
    elif status_result['status'] == 'FAILED':
        print("Job failed!")
        print(status_result)
        break

Job submitted: 5825ba30-0f68-47c0-be63-ac129739581b-u1
Status: IN_QUEUE
Status: IN_QUEUE
Status: IN_QUEUE
Status: IN_QUEUE
Status: IN_QUEUE
Status: IN_QUEUE
Status: IN_QUEUE
Status: IN_QUEUE
Status: IN_PROGRESS
Status: COMPLETED

Final result:
{'delayTime': 16621, 'executionTime': 1037, 'id': '5825ba30-0f68-47c0-be63-ac129739581b-u1', 'output': {'data': [{'embedding': [-0.006651742849498987, -0.0013699487317353487, -0.1715860664844513, 0.008385584689676762, 0.0058329845778644085, 0.0697389468550682, -0.00016455438162665814, -0.04298213869333267, -0.014566407538950443, -0.05402735248208046, 0.0005869775195606053, 0.039407432079315186, 0.027762865647673607, 0.08082697540521622, 0.04542236030101776, -0.06301763653755188, 0.010349534451961517, -0.029732167720794678, -0.0426824651658535, 0.029603734612464905, -0.0036763858515769243, -0.09435521811246872, -0.007497258484363556, 0.03799467161297798, 0.09230029582977295, -0.014309542253613472, -0.014941003173589706, 0.0616476908326149, 0.00646

In [ ]:
import asyncio
import aiohttp
import time
import nest_asyncio

text_list = texts_list  # texts_list[:10]  # Using first 10 items

# Configuration
API_URL = '<api-url>/run'
STATUS_URL = '<api-url>/status/'
API_KEY = '<api-key>'

headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {API_KEY}'
}

async def poll_job_status(session, job_id):
    """Poll job status until completion"""
    while True:
        await asyncio.sleep(2)  # Wait 2 seconds

        async with session.get(f"{STATUS_URL}{job_id}", headers=headers) as response:
            if response.status == 200:
                result = await response.json()

                if result['status'] == 'COMPLETED':
                    return result
                elif result['status'] == 'FAILED':
                    print(f"Job {job_id} failed!")
                    return None
                # Continue polling if still IN_QUEUE or IN_PROGRESS
            else:
                print(f"Error checking status for job {job_id}")
                return None

async def process_text_async(session, text):
    try:
        data = {
            "input": {
                "model": "nomic-ai/nomic-embed-text-v1.5",
                "input": text
            }
        }

        # Submit job
        async with session.post(API_URL, headers=headers, json=data) as response:
            if response.status == 200:
                submit_result = await response.json()
                job_id = submit_result['id']
                print(f"Job submitted for text '{text[:30]}...': {job_id}")

                # Poll for completion
                final_result = await poll_job_status(session, job_id)
                return final_result
            else:
                print(f"HTTP Error {response.status} for text '{text[:50]}...'")
                return None

    except Exception as e:
        print(f"Error processing text '{text[:50]}...': {str(e)}")
        return None

async def main():
    async with aiohttp.ClientSession() as session:
        tasks = [process_text_async(session, text) for text in text_list]
        results = []

        # Simple iteration without tqdm progress bar
        for future in asyncio.as_completed(tasks):
            result = await future
            results.append(result)
            print(f"Completed {len(results)}/{len(text_list)} tasks")

    return results

# In Colab/Jupyter:
nest_asyncio.apply()
strt_time = time.time()
all_results = await main()
print(f"time taken {time.time() - strt_time}")

# Print results summary
successful = [r for r in all_results if r is not None]
print(f"Total: {len(all_results)}, Successful: {len(successful)}")

if successful:
    print(f"\nFirst successful result:")
    print(successful[0])

Job submitted for text 'p2 :
al →blar
p3 :
br →ar
p4 :...': d83b82a5-e9da-4fcd-b5cb-d6385a206e67-u2
Job submitted for text 'portional to the cell volume,
...': 4713f8b9-6e67-4a99-b141-b6f66b870d12-u1
Job submitted for text 'For example, Figure 3.23 prese...': 4928e86b-f35b-4aae-99d9-779be81f0fc3-u1
Job submitted for text 'Production p3 describes the li...': 2efc3cdd-92bc-49ad-a6a3-73b82765f31e-u1
Job submitted for text '1The model of Anabaena introdu...': 69382cad-3edb-44af-8ea7-48f11c75a962-u1
Job submitted for text 'surface of revolution.
Figure ...': ed6c386d-bb02-4a56-a98d-7977530afe8e-u2
Job submitted for text 'the corresponding lowercase le...': 914d5c3d-caa5-4bd9-ab74-b8bd082b7842-u1
Job submitted for text '=

ak+1
bk+1 
1.9. Growth fu...': 20d22126-93b3-4507-b97f-5547eec1171b-u2
Job submitted for text 'ural Sciences and Engineering ...': f0a119ab-de50-4ec3-b27e-8b2e5541ab81-u2
Job submitted for text 'p5). After a delay of d steps ...': c6db8b2b-02fd-40e5-b719-00d566f1f409-u2
